In [1]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

CUDA available: True
Number of GPUs available: 2
GPU 0: NVIDIA GeForce RTX 2070
GPU 1: NVIDIA GeForce RTX 2070
GPU 0 memory allocated: 0.00 GB
GPU 0 memory reserved: 0.00 GB
GPU 1 memory allocated: 0.00 GB
GPU 1 memory reserved: 0.00 GB


In [2]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2H5, FFF2H5, H5, DownloadData

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, CellProperties, DilationedCytoMask ,\
                                    DisplaySequentialParams, IlluminationCorrection, Calculate_BIGFISH_Threshold, rescale_images,\
                                    Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [3]:
scope = ScopeClass() 
data = DataContainer(clear_after_error=False)
settings = Settings(name='demo')
experiment = Experiment()

In [4]:
# you can change the parameters
experiment.initial_data_location = ['smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_0min_20220224']
                                    # 'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_10min_20220224']

# data.local_dataset_location = [r'C:\Users\formanj\GitHub\FISH_Processing\Demos\DUSP1_Dex_10min_20220224', 
#                                  r'C:\Users\formanj\GitHub\FISH_Processing\Demos\DUSP1_Dex_0min_20220224']
experiment.FISHChannel = 0
experiment.nucChannel = 2
experiment.cytoChannel = 1
experiment.voxel_size_z = 500

settings.num_chunks_to_run = 5 # this is the limit to how many chunks you want to run and it will stop after that
# settings.psuedoCyto = 1
settings.sigma_dict = {0: 50, 1: 100, 2: 100}

scope.spot_yx = 130
scope.spot_z = 360
scope.voxel_size_yx = 100

Parameters().get_parameters()

{'state': 'global',
 'instances': [],
 'voxel_size_yx': 100,
 'spot_z': 360,
 'spot_yx': 130,
 'init': True,
 'local_dataset_location': None,
 'total_num_chunks': None,
 'images': None,
 'masks': None,
 'temp': None,
 'clear_after_error': False,
 'mask_locations': None,
 'image_locations': None,
 'name': 'demo',
 'return_data_to_NAS': True,
 'NUMBER_OF_CORES': 4,
 'save_files': True,
 'num_chunks_to_run': 5,
 'connection_config_location': 'c:\\Users\\formanj\\GitHub\\AngelFISH\\config_nas.yml',
 'display_plots': True,
 'load_in_mask': False,
 'mask_structure': {'masks': ('ptczyx', None, None),
  'cell_mask': ('zyx', 'cytoChannel', 'masks'),
  'nuc_mask': ('zyx', 'nucChannel', 'masks')},
 'order': 'pt',
 'share_name': 'share',
 'sigma_dict': {0: 50, 1: 100, 2: 100},
 'initial_data_location': ['smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_0min_20220224'],
 'index_dict': None,
 'nucChannel': 2,
 'cytoChannel': 1,
 'FISHChannel': 0,
 'voxel_size_z': 500,
 'independent_params': [{}],

In [5]:
Parameters.validate()

In [6]:
DownloadData().run()

Overwriting local_dataset_location in DataContainer


{'local_dataset_location': ['c:\\Users\\formanj\\GitHub\\AngelFISH\\dataBases\\DUSP1_Dex_0min_20220224']}

In [7]:
settings.load_in_mask = True
FFF2H5().run()

Traceback (most recent call last):
  File "c:\Users\formanj\GitHub\AngelFISH\src\GeneralStep.py", line 17, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\formanj\GitHub\AngelFISH\src\GeneralStep.py", line 162, in run
    results = self.main(**kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\formanj\GitHub\AngelFISH\src\IndependentSteps\DataTypeBridges.py", line 204, in main
    images, masks, ip, position_indexs, mask_locations = self.load_in_dataset(local_dataset_location,
                                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\formanj\GitHub\AngelFISH\src\IndependentSteps\DataTypeBridges.py", line 236, in load_in_dataset
    masks[key] = [da.from_array(h5[key]) for h5 in h5_files]
                  ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\formanj\GitHub\AngelFISH\.venv\Lib\site-packages\dask\array\core.py", line 3639, in from_array
    chunks, x.shape, dtype=x.dtype, 

AttributeError: 'Group' object has no attribute 'shape'

In [ ]:
# this will load in your data
print(data.load_temp_data())

In [ ]:
# IlluminationCorrection().run()

In [ ]:
Display().displayImage_maxProject(channel=2)

In [ ]:
# settings.sigma_dict = [120, 120, 120]
# IlluminationCorrection().run()

In [ ]:
# if you would like to remove the original data from disk
# remove_local_data_but_keep_h5().run()

In [ ]:
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90]
settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

SimpleCellposeSegmentaion().run()

In [ ]:
# settings.MAX_NUM_IMAGES_TO_AUTOMATICALLY_CALCULATE_THRESHOLD = 50
# Automatic_BIGFISH_Threshold().run()

In [ ]:
print(data)

In [ ]:
settings.verbose = True
# settings.bigfish_threshold = 'mean'
BIGFISH_SpotDetection().run()

In [ ]:
# DilationedCytoMask().run()

In [ ]:
import matplotlib.pyplot as plt

params = Parameters().get_parameters()

nuc = params['masks']['cell'][0,0,0]

plt.imshow(nuc)
plt.show()

In [ ]:
# DisplaySequentialParams().run()

In [ ]:
CellProperties().run()

In [ ]:
# inst[-1].cell_properties.columns

In [ ]:
# print(inst[-1].cell_properties['nuc_label'])
# print(inst[-1].cell_properties['cell_label'])
# print(inst[-1].cell_properties['cyto_label'])


In [ ]:
# Display().displayMask(channel=1, label=5)

In [ ]:
Save_Masks().run()



In [ ]:
Save_Outputs().run()

In [ ]:
data.cellresults.head()

In [ ]:
import h5py

f = h5py.File(r'C:\Users\formanj\GitHub\AngelFISH\dataBases\DUSP1_Dex_0min_20220224.h5', 'a')
# Close the file first
f.close()

# Use pandas to save the dataframe to the HDF5 file
data.cellresults.to_hdf(r'C:\Users\formanj\GitHub\AngelFISH\dataBases\DUSP1_Dex_0min_20220224.h5', 
                       key='test', mode='a')

# Reopen the file
f = h5py.File(r'C:\Users\formanj\GitHub\AngelFISH\dataBases\DUSP1_Dex_0min_20220224.h5', 'a')